In [ ]:
import numpy as np
import tensorflow as tf

: 

In [ ]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [ ]:
# Prepare Data set for Cats and Dogs:

'1. Data preprocessing: Image Generator, Train Test Split'
'2. Model architecting'
'3. Model compile'
'4. Model Traiing'
'5. Model Testing / Input testing'

In [ ]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train_image_data = train_image_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/train',
                                                             class_mode= 'binary',
                                                             target_size = (255, 255),
                                                             batch_size = 10
                                                             )

test_image_data = test_image_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/validation', class_mode='binary',
                                                           target_size = (255, 255),
                                                           batch_size = 10)

In [ ]:

vgg = tf.keras.applications.vgg16.VGG16(include_top=False)
vgg.summary()

In [ ]:
# We want to retain the existing weights of the existing model | We need not to train them:
# This can be archived using: layer.trainble = False

for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# #Create our custom model: In that create Input layer
input_layer = tf.keras.layers.Input(shape=[255,255,3]) # Decleration of Input layer

x = vgg(input_layer) # Now X contains vegg with Input layer

x1= tf.keras.layers.Flatten()(x) # Flatten the Convolution layer ip -> vgg -> flatten

x2 = tf.keras.layers.Dense(activation= 'relu', units = 128, name= 'h1')(x1) # First ANN layer 

x3 = tf.keras.layers.Dense(activation = 'relu', units = 128, name = 'h2')(x2)

output = tf.keras.layers.Dense(activation='sigmoid', units = 1, name = 'output')(x3)

# Combine all (input and output) these layers and then create a model:

model = tf.keras.models.Model(inputs = input_layer, outputs = output)

model.summary()



In [ ]:
model.compile(optimizer= 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
              )

In [ ]:
model.fit(train_image_data,
            validation_data = test_image_data,
            steps_per_epoch= (len(train_image_data.filenames) // train_image_data.batch_size),
            validation_steps= (len(test_image_data.filenames) // test_image_data.batch_size),
            epochs= 10,
            callbacks = [MyCLRuleMonitor(0.9)])

On collab:Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 223ms/step - accuracy: 0.7120 - loss: 1.0210 - val_accuracy: 0.9050 - val_loss: 0.2357
<keras.src.callbacks.history.History at 0x7e60e4995c40>

In [ ]:
# Input | Deployment

img = tf.keras.preprocessing.image.load_img('dd.jpg', target_size = (255,255))
img_arr = tf.keras.preprocessing.image.img_to_array(img)

img_arr.shape

np_img = np.expand_dims(img_arr, axis = 0)
np_img.shape

: 

In [ ]:
# Model Prediction

probability_prediction = model.predict(np_img)

# For binary classification with sigmoid output, we apply a threshold
# If probability_prediction[0][0] >= 0.5, it's class 1 (dogs), otherwise class 0 (cats)
if probability_prediction[0][0] >= 0.5:
    prediction_class_index = 1  # Corresponds to 'dogs'
else:
    prediction_class_index = 0  # Corresponds to 'cats'

print(f"Raw probability: {probability_prediction[0][0]:.4f}")

class_names_map = {v: k for k, v in train_image_data.class_indices.items()}

# Use the index to look up the actual class name
predicted_class_name = class_names_map[prediction_class_index]

# Print the result
print(f"The predicted pet name is: {predicted_class_name}")

On collab it is able to make Prediction